In [1]:
import requests, json, pprint
from datetime import datetime, timedelta

# fetch the results
res = requests.get("https://ct-mock-tech-assessment.herokuapp.com").json()['partners']

class Partner:

    def __init__(self, info):
      self.dates = []
      for d in info['availableDates']:
          self.dates.append(datetime.strptime(d, '%Y-%m-%d'))
      self.country = info['country']
      self.email = info['email']
      self.firstName = info['firstName']
      self.lastName = info['lastName']

    def __repr__(self):
      return f"""Name: {self.firstName} {self.lastName}
Email: {self.email}\n"""

class Conference:
    
    def __init__(self, country, parts):
        self.attendeeCount = 0
        self.attendees = []
        self.startDate = ""
        self.country = country
        self.potential_attendees = parts[country]
        self.potential_dates = set()
    
    def find_best_date(self):
        for p in self.potential_attendees:
            for d in p.dates:
                self.potential_dates.add(d)
        self.potential_dates = list(self.potential_dates)
        self.potential_dates.sort()
        for d in self.potential_dates:
            count = 0
            atts = []
            for p in self.potential_attendees:
                if d and (d + timedelta(days = 1)) in p.dates:
                    count += 1
                    atts.append(p)
            if count > self.attendeeCount:
                self.attendeeCount = count
                self.startDate = d
                self.attendees = atts
        return {
            'attendeeCount': self.attendeeCount,
            'attendees': [a.email for a in self.attendees],
            'name': self.country,
            'startDate': self.startDate.strftime('%Y-%m-%d')
        }

class Organizer:
    def __init__(self, data=None):
        self.partners = {}
        self.conferences = []
        self.final_conf_data = []
        for i in data:
            if i["country"] not in self.partners.keys():
                self.partners[i["country"]] = []
            self.partners[i["country"]].append(Partner(i))

    def run(self):
        for k in self.partners.keys():
            self.conferences.append(Conference(k, self.partners))

        for c in self.conferences:
            self.final_conf_data.append(c.find_best_date())
        return self.final_conf_data
    def send(self):
        r = requests.post("https://ct-mock-tech-assessment.herokuapp.com", json=self.final_conf_data)
        print(r.status_code)
       
org = Organizer(res)
org.run()
print(org.final_conf_data)

org.send()

[{'attendeeCount': 16, 'attendees': ['rketring@codingtemple.com', 'ccafaro@codingtemple.com', 'mlacau@codingtemple.com', 'eomersa@codingtemple.com', 'ssheahan@codingtemple.com', 'msoult@codingtemple.com', 'bleeks@codingtemple.com', 'lsolid@codingtemple.com', 'bwatton@codingtemple.com', 'jmasher@codingtemple.com', 'afernadez@codingtemple.com', 'mmottern@codingtemple.com', 'pdougherty@codingtemple.com', 'sdegunya@codingtemple.com', 'sbeemon@codingtemple.com', 'mvirgel@codingtemple.com'], 'name': 'United States', 'startDate': '2017-05-28'}, {'attendeeCount': 15, 'attendees': ['sterrence@codingtemple.com', 'dpatnode@codingtemple.com', 'enawwar@codingtemple.com', 'pbertog@codingtemple.com', 'rguldemond@codingtemple.com', 'rfarlee@codingtemple.com', 'elaskin@codingtemple.com', 'greif@codingtemple.com', 'mclawson@codingtemple.com', 'mbottenfield@codingtemple.com', 'akulju@codingtemple.com', 'cscoby@codingtemple.com', 'dsiebenthal@codingtemple.com', 'dharriet@codingtemple.com', 'mcrossen@codin